### QR-декомпозиция
##### Задача:
Написать QR-декомпозицию матрицы руками.

Напомним, что QR-разложение - это представление произвольной матрицы в виде произведения ортогональной Q (т.е. такой, что обратная совпадает с транспонированной, $Q^{-1} = Q^T$) и верхне-треугольной R (т.е. такой, что все элементы ниже главной диагонали - нули).

Будем строить QR-разложение черз процесс ортогонализации Грамма-Шмидта ([wiki](https://en.wikipedia.org/wiki/QR_decomposition#Using_the_Gram%E2%80%93Schmidt_process)).
Для этого рассмотрим изначальную матрицу A размера $m\times n$, как набор векторов-столбцов и применим к ним процесс ортогонализации.
Полученный в результате ортонормированный базис запишем по столбцам в матрицу, это и будет Q.
R найдем как $R = Q^T A$ ($A = QR$, домножая на $Q^T$ слева получаем выражение выше).

Если исходная матрица прямоугольная (или неполного ранга), то есть несколько изменений алгоритма.
Во-первых, если на каком-либо этапе ортогонализации столбец из матрицы A не является линейно независимым (от уже выбранных столбцов матрицы Q), то в результате ортогонализации получится нулевой вектор и его мы не добавляем.
Во-вторых, если $rk A < m$, то ортогонализация даст недостаточно вектор-столбцов, поэтому мы в дополнеие берем случайные вектора, ортогонализируем их и добавляем к матрице Q.


In [87]:
import numpy as np

def qr_factorization(a: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Compute the QR factorization of a matrix."""
    m, n = a.shape
    q = np.empty((m, 0))

    i = 0
    while q.shape[1] < m:
        # iterate over columns of original matrix. Use random vectors if matrix A is rectangular and n < m
        ai = a[:, i] if i < n else np.random.rand(m)
        ui = ai
        for j in range(q.shape[1]):
            # subtract projections of ai on all existing vectors of Q
            qj = q[:, j]
            proj = np.dot(ai, qj) * qj
            ui = ui - proj
        # at this point vector ui is orthogonal to every column-vector of Q
        # if ai on current iteration is linear combination of existing ei, ui will be close to zero
        # therefore we do not include it in matrix Q
        if not np.allclose(np.zeros_like(ui), ui):
            # normalize ui
            ei = ui / np.linalg.norm(ui)
            q = np.hstack((q, ei.reshape(-1, 1)))
        i += 1

    r = np.dot(q.T, a)
    return q, r

def qr_check(a, q, r) -> bool:
    """Check if a = q * r, q is orthogonal, r is upper triangular."""
    # is a = q * r
    if not np.allclose(a, np.dot(q, r)):
        print("a != q * r")
        return False
    # is q orthogonal
    if not np.all(np.isclose(np.identity(q.shape[0]), np.dot(q.T, q))):
        print("q is not orthogonal")
        return False
    # is r upper triangular
    if not np.allclose(r, np.triu(r)):
        print("r is not upper triangular")
        return False
    return True


Ниже есть довольно произвольный список матриц (и их разложений) для теста вышенаписанного кода.

In [103]:
sample_matrices = [
    np.array([[1]]),
    np.array([[1, 1]]),
    np.array([[1], [1]]),
    np.array([[1, 2], [1, 0]]),
    np.array([[1, 2, 3], [3, 2, 1]]),
    np.array([[1, 3], [2, 2], [3, 1]]),
    np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])
]
with np.printoptions(precision=3, suppress=True):
    for a in sample_matrices:
        q, r = qr_factorization(a)
        if qr_check(a, q, r):
            print("A ~~~\n", a, "\nQ ~~~\n", q, "\nR ~~~\n", r, "\n\n\n")

A ~~~
 [[1]] 
Q ~~~
 [[1.]] 
R ~~~
 [[1.]] 



A ~~~
 [[1 1]] 
Q ~~~
 [[1.]] 
R ~~~
 [[1. 1.]] 



A ~~~
 [[1]
 [1]] 
Q ~~~
 [[ 0.707  0.707]
 [ 0.707 -0.707]] 
R ~~~
 [[1.414]
 [0.   ]] 



A ~~~
 [[1 2]
 [1 0]] 
Q ~~~
 [[ 0.707  0.707]
 [ 0.707 -0.707]] 
R ~~~
 [[1.414 1.414]
 [0.    1.414]] 



A ~~~
 [[1 2 3]
 [3 2 1]] 
Q ~~~
 [[ 0.316  0.949]
 [ 0.949 -0.316]] 
R ~~~
 [[3.162 2.53  1.897]
 [0.    1.265 2.53 ]] 



A ~~~
 [[1 3]
 [2 2]
 [3 1]] 
Q ~~~
 [[ 0.267  0.873  0.408]
 [ 0.535  0.218 -0.816]
 [ 0.802 -0.436  0.408]] 
R ~~~
 [[ 3.742  2.673]
 [-0.     2.619]
 [-0.    -0.   ]] 



A ~~~
 [[ 12 -51   4]
 [  6 167 -68]
 [ -4  24 -41]] 
Q ~~~
 [[ 0.857 -0.394 -0.331]
 [ 0.429  0.903  0.034]
 [-0.286  0.171 -0.943]] 
R ~~~
 [[ 14.  21. -14.]
 [ -0. 175. -70.]
 [ -0.  -0.  35.]] 



